<a href="https://colab.research.google.com/github/pokjay/heb-squad/blob/main/finetune_alephbert_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Aug 24 23:23:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/pokjay/heb-squad

Cloning into 'heb-squad'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 64 (delta 19), reused 36 (delta 5), pack-reused 0
Unpacking objects: 100% (64/64), done.
Checking out files: 100% (21/21), done.


In [ ]:
!gzip -d /content/heb-squad/data/final/heb-train-v2.0.csv.gz
!gzip -d /content/heb-squad/data/final/heb-dev-v2.0.csv.gz

In [ ]:
pip install transformers

     |████████████████████████████████| 2.6 MB 8.8 MB/s 
     |████████████████████████████████| 3.3 MB 36.1 MB/s 
     |████████████████████████████████| 895 kB 44.2 MB/s 
     |████████████████████████████████| 636 kB 63.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import collections
import pandas as pd
from tqdm.notebook import trange, tqdm

import torch
from torch.utils.data import DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
BATCH_SIZE = 16

In [ ]:
df = pd.read_csv('/content/heb-squad/data/final/heb-train-v2.0.csv')
val_df = pd.read_csv('/content/heb-squad/data/final/heb-dev-v2.0.csv')

In [ ]:
df = df[(df.answer_start_heb > 0) & (df.answer_end_heb > 0)]
val_df = val_df[(val_df.answer_start_heb > 0) & (val_df.answer_end_heb > 0)]

Remove answer for impossible questions (The answer is actually the plausible answer)

In [ ]:
df['plausible_answer'] = df['answer']
df['plausible_answer_start_heb'] = df['answer_start_heb']
df['plausible_answer_end_heb'] = df['answer_end_heb']

val_df['plausible_answer'] = val_df['answer']
val_df['plausible_answer_start_heb'] = val_df['answer_start_heb']
val_df['plausible_answer_end_heb'] = val_df['answer_end_heb']

In [ ]:
df.loc[df.is_impossible == 1, 'answer'] = ''
df.loc[df.is_impossible == 1, 'answer_start_heb'] = 0
df.loc[df.is_impossible == 1, 'answer_end_heb'] = 0

val_df.loc[val_df.is_impossible == 1, 'answer'] = ''
val_df.loc[val_df.is_impossible == 1, 'answer_start_heb'] = 0
val_df.loc[val_df.is_impossible == 1, 'answer_end_heb'] = 0

In [ ]:
df.sample(3)

,id,context,question,answer,answer_start,answer_end,is_impossible,article,context_marked,answer_start_heb,answer_end_heb,plausible_answer,plausible_answer_start_heb,plausible_answer_end_heb
49169,5a7a3c9517ab25001a8a03f1,ממש מתחת לדירות המדינה יש סוויטה של ​​חדרים קצ...,איזה חדר אינו מאפשר ביקורים של הצאר ניקולאס הר...,,325,334,1,Buckingham_Palace,ממש מתחת לדירות המדינה יש סוויטה של ​​חדרים קצ...,0,0,חדר 1844,232,236
26736,56fdfa85761e401900d28c86,I/O הוא האמצעי שבו מחשב מחליף מידע עם העולם הח...,עכבר הוא איזה סוג של מכשיר היקפי?,קֶלֶט,212,217,0,Computer,I/O הוא האמצעי שבו מחשב מחליף מידע עם העולם הח...,144,147,קֶלֶט,144,147
120093,5734173b4776f41900661850,"ממזרח למרווח נהר מיזורי, שנוצר על ידי מפגש הנה...",ליד היכן נוצרות הנהרות להתמזגות נהר מיזורי?,שלוש מזלגות,124,135,0,Montana,"ממזרח למרווח נהר מיזורי, שנוצר על ידי מפגש הנה...",78,89,שלוש מזלגות,78,89


## Transform the texts to encodings

In [ ]:
train_ids = df.id.to_list()
train_contexts = df.context.to_list()
train_questions = df.question.to_list()

val_ids = val_df.id.to_list()
val_contexts = val_df.context.to_list()
val_questions = val_df.question.to_list()

In [ ]:
train_answers = df.apply(lambda x: {'answer_start' : x.answer_start_heb, 'answer_end': x.answer_end_heb}, axis=1).to_list()
val_answers = val_df.apply(lambda x: {'answer_start' : x.answer_start_heb, 'answer_end': x.answer_end_heb}, axis=1).to_list()

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('onlplab/alephbert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

Downloading:   0%|          | 0.00/545k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        if answers[i]['answer_end'] > 0:
          end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        else:
          end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
train_encodings.update({'id': train_ids})
val_encodings.update({'id': val_ids})

## Create PyTorch dataset

In [ ]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        # Get all encoded vals as tensors
        vals = {key: torch.tensor(val[idx]) for key, val in self.encodings.items() if key != 'id'}
        # Add the id which is a string, used to map predictions to ids later
        vals.update({key: val[idx] for key, val in self.encodings.items() if key == 'id'})
        return vals

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Import Hebrew BERT for Q&A fine tuning

In [ ]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained("onlplab/alephbert-base")

Downloading:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of the model checkpoint at onlplab/alephbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initi

In [ ]:
def load_checkpoint(path, model, optimizer):

  # Load checkpoint from Google Drive
  checkpoint = torch.load(path)

  # Load checkpoint to model and optimizer
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  checkpoint_epoch = checkpoint['epoch']

  return checkpoint_epoch

## Code to evaluate model

In [ ]:
def calc_metrics(model_inf, dataloader):
  """
  Given a model and a dataset calculate the following metrics:
  - Exact Match
  - F1 Score
  - Loss
  @returns (F1, EM, Loss, EM Scores, F1 Scores)
  """

  model.to(device)
  model.eval()

  f1_scores = {}
  exact_scores = {}

  loss = 0
  total_cnt = 0

  for counter, batch in tqdm(enumerate(dataloader)):
    with torch.no_grad():

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      ids = batch['id']
      outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      loss += outputs[0]

      pred_start_positions = torch.argmax(outputs['start_logits'], axis=1)
      pred_end_positions = torch.argmax(outputs['end_logits'], axis=1)

      # Calc scores
      for i in range(len(input_ids)):

        total_cnt += 1

        # Check if we have an exact match
        if start_positions[i] == pred_start_positions[i] and end_positions[i] == pred_end_positions[i]:
          # We have an exact match, mark it
          exact_scores[ids[i]] = 1
        else:
          # No match, mark with 0 or keep previous marking
          exact_scores[ids[i]] = max(0, exact_scores.get(ids[i], 0))

        # Get the predicted answer token sequence
        pred_tokens = input_ids[i][pred_start_positions[i]:pred_end_positions[i] + 1].tolist()
        answer_tokens = input_ids[i][start_positions[i]:end_positions[i] + 1].tolist()

        # Check how many predicted tokens correspond to gold answer tokens
        common = collections.Counter(answer_tokens) & collections.Counter(pred_tokens)
        num_same = sum(common.values())

        # If none same, then F1=0 , else use the F1 formula
        if num_same == 0:
          score = 0
        else:
          precision = 1.0 * num_same / len(pred_tokens)
          recall = 1.0 * num_same / len(answer_tokens)
          score = (2 * precision * recall) / (precision + recall)
        
        # We take the max F1 score of the gold answers
        f1_scores[ids[i]] = max(score, f1_scores.get(ids[i], 0))

  f1 = sum(f1_scores.values()) / len(f1_scores)
  em = sum(exact_scores.values()) / len(exact_scores)
  epoch_avg_loss = loss / total_cnt

  return (f1, em, epoch_avg_loss, exact_scores, f1_scores)

## Train model

In [ ]:
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

optim = AdamW(model.parameters(), lr=5e-5)

train_loss = []
val_loss = []

start_epoch = 0

checkpoint_path = '/content/drive/MyDrive/nlp/checkpoints/checkpoint_5_0.398_0.309_0.515_0.207'
start_epoch = load_checkpoint(checkpoint_path, model, optim) + 1

print(f'Training model from epoch={start_epoch}')

for epoch in range(start_epoch, 9):

    model.train()

    epoch_loss = 0

    for counter, batch in enumerate(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        epoch_loss += loss

        if counter % 250 == 0 and counter > 0:
            print('Epoch %d, %d/%d, Current Loss = %.4f' % (epoch, counter, len(train_loader), loss))

    avg_train_loss = epoch_loss / len(train_loader)

    print('Epoch %d, Train Loss = %.4f' % (epoch, avg_train_loss))

    f1, em, avg_val_loss, _, _ = calc_metrics(model, val_loader)

    print('Epoch %d, Val Loss = %.4f, F1 = %.4f, EM=%.4f' % (epoch, avg_val_loss, f1, em))

    train_loss.append(avg_train_loss)
    val_loss.append(avg_val_loss)

    state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'optimizer': optim.state_dict(),
        'f1': f1,
        'em': em,
        'avg_val_loss': avg_val_loss,
        'avg_train_loss': avg_train_loss
    }
    savepath=f'/content/drive/MyDrive/nlp/checkpoints/checkpoint_{epoch}_{f1:.3f}_{em:.3f}_{avg_train_loss:.3f}_{avg_val_loss:.3f}'
    torch.save(state, savepath)

Training model from epoch=6
Epoch 6, 250/7113, Current Loss = 0.3439
Epoch 6, 500/7113, Current Loss = 0.5555
Epoch 6, 750/7113, Current Loss = 0.0742
Epoch 6, 1000/7113, Current Loss = 0.4545
Epoch 6, 1250/7113, Current Loss = 0.1283
Epoch 6, 1500/7113, Current Loss = 0.1669
Epoch 6, 1750/7113, Current Loss = 0.1741
Epoch 6, 2000/7113, Current Loss = 0.6775
Epoch 6, 2250/7113, Current Loss = 0.2455
Epoch 6, 2500/7113, Current Loss = 0.2848
Epoch 6, 2750/7113, Current Loss = 0.6664
Epoch 6, 3000/7113, Current Loss = 0.6268
Epoch 6, 3250/7113, Current Loss = 0.4088
Epoch 6, 3500/7113, Current Loss = 0.3085
Epoch 6, 3750/7113, Current Loss = 0.1559
Epoch 6, 4000/7113, Current Loss = 0.1712
Epoch 6, 4250/7113, Current Loss = 0.4602
Epoch 6, 4500/7113, Current Loss = 0.3085
Epoch 6, 4750/7113, Current Loss = 0.4058
Epoch 6, 5000/7113, Current Loss = 0.3633
Epoch 6, 5250/7113, Current Loss = 0.3022
Epoch 6, 5500/7113, Current Loss = 0.2693
Epoch 6, 5750/7113, Current Loss = 0.3785
Epoch 6, 

0it [00:00, ?it/s]

Epoch 6, Val Loss = 0.1562, F1 = 0.5456, EM=0.4838
Epoch 7, 250/7113, Current Loss = 0.5024
Epoch 7, 500/7113, Current Loss = 0.0346
Epoch 7, 750/7113, Current Loss = 0.4266
Epoch 7, 1000/7113, Current Loss = 0.0792
Epoch 7, 1250/7113, Current Loss = 0.0646
Epoch 7, 1500/7113, Current Loss = 0.3357
Epoch 7, 1750/7113, Current Loss = 0.4848
Epoch 7, 2000/7113, Current Loss = 0.1134
Epoch 7, 2250/7113, Current Loss = 0.4472
Epoch 7, 2500/7113, Current Loss = 0.5040
Epoch 7, 2750/7113, Current Loss = 0.3859
Epoch 7, 3000/7113, Current Loss = 0.3360
Epoch 7, 3250/7113, Current Loss = 0.5983
Epoch 7, 3500/7113, Current Loss = 0.1100
Epoch 7, 3750/7113, Current Loss = 0.1533
Epoch 7, 4000/7113, Current Loss = 0.1970
Epoch 7, 4250/7113, Current Loss = 0.1663
Epoch 7, 4500/7113, Current Loss = 0.0600
Epoch 7, 4750/7113, Current Loss = 0.1953
Epoch 7, 5000/7113, Current Loss = 0.2800
Epoch 7, 5250/7113, Current Loss = 0.1105
Epoch 7, 5500/7113, Current Loss = 0.4141
Epoch 7, 5750/7113, Current 

0it [00:00, ?it/s]

Epoch 7, Val Loss = 0.1623, F1 = 0.5222, EM=0.4598
Epoch 8, 250/7113, Current Loss = 0.1074
Epoch 8, 500/7113, Current Loss = 0.5580
Epoch 8, 750/7113, Current Loss = 0.1567
Epoch 8, 1000/7113, Current Loss = 0.1489
Epoch 8, 1250/7113, Current Loss = 0.2828
Epoch 8, 1500/7113, Current Loss = 0.1840
Epoch 8, 1750/7113, Current Loss = 0.2435
Epoch 8, 2000/7113, Current Loss = 0.1582
Epoch 8, 2250/7113, Current Loss = 0.1789
Epoch 8, 2500/7113, Current Loss = 0.2230
Epoch 8, 2750/7113, Current Loss = 0.3909
Epoch 8, 3000/7113, Current Loss = 0.0668
Epoch 8, 3250/7113, Current Loss = 0.1791
Epoch 8, 3500/7113, Current Loss = 0.0902
Epoch 8, 3750/7113, Current Loss = 0.6656
Epoch 8, 4000/7113, Current Loss = 0.3748
Epoch 8, 4250/7113, Current Loss = 0.3490
Epoch 8, 4500/7113, Current Loss = 0.5590
Epoch 8, 4750/7113, Current Loss = 0.0598
Epoch 8, 5000/7113, Current Loss = 0.2993
Epoch 8, 5250/7113, Current Loss = 0.4242
Epoch 8, 5500/7113, Current Loss = 0.2514
Epoch 8, 5750/7113, Current 

0it [00:00, ?it/s]

Epoch 8, Val Loss = 0.1747, F1 = 0.5297, EM=0.4641
